In [184]:
import pandas
from pandas import DataFrame
from altair import Chart, Color, Tooltip, X, Y

from app.data import MongoDB


In [159]:
def pie_chart(df, column, title):
    chart = Chart(
        df,
        title=title,
    ).mark_arc(innerRadius=80).encode(
        color=Color(column),
        theta="count()",
        tooltip=Tooltip([column, "count()"]),
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )
    return chart

In [160]:
DB = MongoDB()


In [161]:
mentors = DB.projection("Mentors", {
    "validate_status": "approved",
    "is_active": True,
    "accepting_new_mentees": True,
}, {"tech_stack": True})


In [162]:
mentors = DataFrame(mentors)


In [163]:
mentors = mentors.explode(ignore_index=True, column="tech_stack")


In [164]:
pie_chart(mentors, "tech_stack", "Mentor Tech Stacks")


alt.Chart(...)

In [165]:
mentees = DB.projection("Mentees", {
    "is_active": True,
    "validate_status": "approved",
}, {"tech_stack": True})


In [166]:
mentees = DataFrame(mentees)


In [167]:
pie_chart(mentees, "tech_stack", "Mentee Tech Stacks")


alt.Chart(...)

### Stacked Bar Chart

In [197]:
# Data Source
DB = MongoDB()

# Mentors
mentors = DB.projection("Mentors", {
    "validate_status": "approved",
    "is_active": True,
    "accepting_new_mentees": True,
}, {"tech_stack": True})
mentors_df = DataFrame(mentors)
mentors_df = mentors_df.explode(column="tech_stack", ignore_index=True)
mentors_df["role"] = ["Mentor"] * mentors_df.shape[0]

# Mentees
mentees = DB.projection("Mentees", {
    "is_active": True,
    "validate_status": "approved",
}, {"tech_stack": True})
mentees_df = DataFrame(mentees)
mentees_df["role"] = ["Mentee"] * mentees_df.shape[0]

# Combined DFs
df = pandas.concat([mentees_df, mentors_df], ignore_index=True)

In [205]:
def bar_chart(x, y, z, title):
    return Chart(
        df,
        title=title,
    ).mark_bar().encode(
        x=X(x, title=x.replace("_", " ").title(), sort="-y"),
        y=Y(y, title=y.replace("_", " ").title()),
        color=Color(z, title=z.replace("_", " ").title()),
        tooltip=Tooltip([z, x, y]),
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )

In [206]:
bar_chart(
    x="tech_stack",
    y="count(tech_stack)",
    z="role",
    title="Tech Stack Count by Role",
)

alt.Chart(...)